This notebook assumes access to denoised, detrended signals and calculates the appropriate parameters for the signal.
This notebook assumes access to detrended, denoised signals and calculates various parameters (and uncertainties) from these signals.The parameters estimated within this notebook are split into three different types: time-domain, frequency-domain and non-linear:

Time domain:
 - mean RR intervals
 - std RR intervals 
 - mean amplitude
 - RMSSD of differences between successive RR intervals 
 - pNN50 (%) NN50 divided by total number of RR (NN50 is the number of sucessive RR intervals that differ by more than 50ms) 
 
 
Frequency Domain:
- absolute power of LF (0.04 - 0.15 Hz) band
- absolute power of HF (0.15 to 0.4Hz) band
- LF/HF ratio
- Total power (0-0.4 Hz)

Non-linear:
- Fractal dimension of dynamic attractor of signal
- std of poincare
- Shannon entropy
- recurrence rate
- detrended fluctuation analysis
- correlation dimension
- sample entropy


In [1]:
%run "Data_PreProcessing.ipynb" #allowing access to the filtered database with preprocessed signals

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:02<00:00, 91.84it/s]


In [2]:
from scipy.signal import find_peaks #for peak finding

In [3]:
def get_rri(signal):
    """
    func that takes in the pre-processed ECG signal and returns the R-R intervals
    """
    peaks, _ = find_peaks(signal, distance = 600)
    peak_to_peaks = peaks[1:-1]

    # calculates rr intervals
    rri = np.diff(peak_to_peaks)
    return rri

### Time Domain

In [4]:
def RR_analysis(signal):
    peak_distances = get_rri(signal)
    
    # stastitical analysis on rr intervals
    mean_RR = np.mean(peak_distances)
    std_RR = np.std(peak_distances)

    #RMSSD
    # computes differences between successive RR intervals for RMSSD
    diff_RR_intervals = np.diff(peak_distances)
    
    RMSSD_RR = np.sqrt(np.mean(diff_RR_intervals**2))
    
    #pNN50
    # the number of successive RR intervals that differ by more than 50 ms 
    NN50 = np.sum(np.abs(diff_RR_intervals) > 50)
    # divide by total number of RR intervals
    pNN50 = (NN50 / len(peak_distances)) * 100
    
    return mean_RR, std_RR, RMSSD_RR, pNN50


In [8]:
rr_means = np.zeros(shape = no_patients)
rr_stds = np.zeros(shape = no_patients)
rr_RMSSD = np.zeros(shape = no_patients)
rr_pNN50s = np.zeros(shape = no_patients)
for i, signal in enumerate(denoised_signals):
    rr_mean, rr_std, rr_rmssd, rr_pNN50 = RR_analysis(signal)
    rr_means[i] = rr_mean
    rr_stds[i] = rr_std
    rr_RMSSD[i] = rr_rmssd
    rr_pNN50s[i] = rr_pNN50

In [9]:

#create dictionary with parameters
params = {}

params['rr_mean'] = rr_means
params['rr_std'] = rr_stds
params['RMSSD'] = rr_RMSSD
params['pNN50'] = rr_pNN50s

